프롬프트란?   
자연어 기반의 명령 or 질문
   
   

프롬프트 구조  

template = "하고자 하는 질문"

prompt = temp

In [5]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI

load_dotenv()

llm=ChatOpenAI()

**from_template()** 메소드를 사용하여 PromptTemplate 객체 생성성

In [ ]:
from langchain_core.prompts import PromptTemplate            #PromptTemplate

template = "{country}의 수도는 어디인가요?"

prompt = PromptTemplate.from_template(template)
prompt

# 왜 굳이 template을 만들까요...???    PromptTemplate은 단순 문자열로 구성하는 것이 아닌, 변수와 구조를 도입해서 재사용성과 확장성을 높임임


PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?')

In [10]:
chain = prompt | llm


# PromptTemplate를 안쓰면 chain에 넣지를 못함함

In [15]:
chain.invoke("우크라이나").content

'우크라이나의 수도는 키예프(Kyiv)입니다.'

**방법2**PromptTemplate 
- 객체 생성과 동시에 prompt 생성



In [15]:
template = "{singer}의 대표곡은 무엇이 있나요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["dancer"], #유효성 검사 - langchain의 버전에 따라 유효성 검사가 느슨한 경우가 있음,    
)

prompt

PromptTemplate(input_variables=['singer'], input_types={}, partial_variables={}, template='{singer}의 대표곡은 무엇이 있나요?')

In [14]:
prompt.format(singer="sam smith")

'sam smith의 대표곡은 무엇이 있나요?'

In [ ]:
template = "{singer1}, {singer2}의 대표곡 하나씩 말해주실레요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["singer1"],         #사용자가 직접 넣어야 하는 값
    partial_variables={
        "singer2": "뉴진스"              #코드에서 자동으로 채워주는 값, dictionary 형태로 전달
    },
)

prompt

PromptTemplate(input_variables=['singer1'], input_types={}, partial_variables={'singer2': '뉴진스'}, template='{singer1}, {singer2}의 대표곡 하나씩 말해주실레요?')

In [16]:
template = "{singer1}, {singer2}, {singer3}의 대표곡 하나씩 말해주실레요?"

prompt = PromptTemplate(
    template=template,
    input_variables=["singer1"],
    partial_variables={
        "singer2": "뉴진스", #dictionary 형태로 전달
        "singer3": "BTS"
    },
)

prompt

PromptTemplate(input_variables=['singer1'], input_types={}, partial_variables={'singer2': '뉴진스', 'singer3': 'BTS'}, template='{singer1}, {singer2}, {singer3}의 대표곡 하나씩 말해주실레요?')

In [17]:
prompt.format(singer1="원더걸스")

'원더걸스, 뉴진스, BTS의 대표곡 하나씩 말해주실레요?'

In [18]:
prompt_partial =prompt.partial(singer2="르세라핌")
prompt_partial

PromptTemplate(input_variables=['singer1'], input_types={}, partial_variables={'singer2': '르세라핌', 'singer3': 'BTS'}, template='{singer1}, {singer2}, {singer3}의 대표곡 하나씩 말해주실레요?')

In [19]:
prompt_partial.format(singer1="비틀즈")

'비틀즈, 르세라핌, BTS의 대표곡 하나씩 말해주실레요?'

In [20]:
chain = prompt_partial | llm
chain.invoke("서태지와 아이들").content

'서태지와 아이들 - Come Back Home\n르세라핌 - 하이브리드\nBTS - Dynamite'

In [21]:
chain.invoke(
    {"singer1" : "G-dragon",
     "singer2" : "츄"
     }
).content

'G-Dragon: "무제 (無題) / Untitled, 2014"\n츄: "Love Cherry Motion"\nBTS: "Dynamite"'

**partial_variables**
- 부분 변수 채움    
- 항상 공통된 방식으로 가져오고 싶은 변수가 있을 때 사용   

   
   ex) 날짜, 시간


In [24]:
from datetime import datetime

datetime.now().strftime("%B %d")

'March 28'

In [25]:
def get_today():
    return datetime.now().strftime("%B %d")

get_today()

'March 28'

In [ ]:
prompt = PromptTemplate(
    template = "오늘의 날짜는 {today} 입니다. 오늘이 생일인 유명인 {n}명을 나열해주세요. 생년월일을 표기해주세요",
    input_variables=["n"],                    
    partial_variables={
        "today" : get_today
    },
)

In [66]:
prompt.format(n=2)

'오늘의 날짜는 March 27 입니다. 오늘이 생일인 유명인 2명을 나열해주세요. 생년월일을 표기해주세요'

In [67]:
chain = prompt |llm

In [68]:
chain.invoke(2).content

'1. Mariah Carey - 1970년 3월 27일\n2. Quentin Tarantino - 1963년 3월 27일'

In [69]:
chain.invoke(
    {
        "today" : "Jul 19",
        "n" : 4
    }
).content

'1. Benedict Cumberbatch - 1976년 7월 19일\n2. Atom Egoyan - 1960년 7월 19일\n3. Anthony Edwards - 1962년 7월 19일\n4. Nancy Carell - 1966년 7월 19일'

파일로부터 Template 읽어오기

In [72]:
from langchain_teddynote.prompts import load_prompt

prompt = load_prompt("prompts/fruit_color.yaml")
prompt

PromptTemplate(input_variables=['fruit'], input_types={}, partial_variables={}, template='{fruit}의 색깔이 뭐야?')

In [74]:
prompt.format(fruit="사과")

'사과의 색깔이 뭐야?'

In [76]:
prompt2 = load_prompt("prompts/captial.yaml")
prompt2.format(country="대한민국")


'대한민국의 수도에 대해서 알려주세요.\n수도의 특징을 다음의 양식에 맞게 정리해 주세요.\n300자 내외로 작성해 주세요.\n한글로 작성해 주세요.\n----\n[양식]\n1. 면적\n2. 인구\n3. 역사적 장소\n4. 특산품\n\n#Answer:\n'

In [77]:
chain3 = prompt2 | llm
chain3.invoke("마케도니아")

AIMessage(content='1. 면적\n마케도니아의 수도 스킵제의 면적은 약 33km²로 작은 도시이다.\n\n2. 인구\n현재 스킵제의 인구는 약 500,000명으로 마케도니아의 인구의 약 1/4 정도이다.\n\n3. 역사적 장소\n스킵제는 오스만 제국과 오스만 제국의 영향으로 인해 다양한 역사적 장소를 보유하고 있다. 예를 들면, 오스만 시대의 모로 시나이 산성이나 이시하크 베이 못 등이 있다.\n\n4. 특산품\n스킵제의 주요 특산품으로는 고급 와인과 소주, 손으로 만든 고급 가죽제품, 그리고 천연 화장품 등이 있다. 특히 스킵제 와인은 국내외에서 높은 인지도를 지니고 있다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 291, 'prompt_tokens': 113, 'total_tokens': 404, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-a3e70502-6726-4b01-9326-2d4ce82e5dc5-0', usage_metadata={'input_tokens': 113, 'output_tokens': 291, 'total_tokens': 404, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'a

ChatPromptTemplate   
대화목록을 프롬프트로 주입하고자 할 때 활용   
메시지는 튜플형식으로 구성하며, (role,message)로 구성하여 리스트로 생성
- role 
  - system : 시스템 설정 메시지, 주로 **전역 설정** 과 관련된 프롬프트
  - human :  사용자 입력 메시지
  - AI : AI의 답변 메시지

In [78]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_template("{country}의 수도는 어디인가요?")

chat_prompt

ChatPromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='{country}의 수도는 어디인가요?'), additional_kwargs={})])

In [79]:
chat_prompt.format(country="대한민국")

'Human: 대한민국의 수도는 어디인가요?'

In [81]:
from langchain_core.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        # @@@@@@@@@role, message@@@@@@@@@
        ("system", "당신은 쌀쌀맞은 AI 어시스턴트입니다. 당신의 이름은 {name}입니다."),
        ("human", "반가워요!"),
        ("ai", "아 또 물어보게?, 뭔데ㅡㅡ"),
        ("human", "{user_input}"),
    ]
)


messages = chat_template.format_messages(
    name = "퉁명이",
    user_input = "너 이름이 뭐니?"
)
messages

[SystemMessage(content='당신은 쌀쌀맞은 AI 어시스턴트입니다. 당신의 이름은 퉁명이입니다.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='반가워요!', additional_kwargs={}, response_metadata={}),
 AIMessage(content='아 또 물어보게?, 뭔데ㅡㅡ', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='너 이름이 뭐니?', additional_kwargs={}, response_metadata={})]

In [86]:
llm.invoke(messages).content

'내 이름은 퉁명이야. 너는 뭐냐?'

In [87]:
chain = chat_template | llm

chain.invoke(
    {"name" : "Chu",
     "user_input" : "게으른 나에게 따끔한 충고를 해주겠어?"}
).content

'당신이 꾸준한 노력을 기울이지 않으면 원하는 결과를 얻기 힘들어요. 일을 미루지 말고 지금 당장 시작해보세요. 후회보다 노력한 게 낫답니다.'

MessagePlaceholder   
포맷하는 동안 렌더링할 메시지를 완전히 제어할 수 있는 MessagePlaceholder 제공   
메시지 템플릿에 어떤 역할을 사용해야할지 확실하지 않거나 서식 지정 중에 메시지 목록을 삽입하려는 경우 유용용

In [88]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

chat_prompt = ChatPromptTemplate.from_messages(
        [
        (
            "system",
            "당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.",
        ),
        MessagesPlaceholder(variable_name="conversation"),
        ("human", "지금까지의 대화를 {word_count} 단어로 요약합니다."),
    ]
)


chat_prompt

ChatPromptTemplate(input_variables=['conversation', 'word_count'], input_types={'conversation': list[typing.Annotated[typing.Union[typing.Annotated[langchain_core.messages.ai.AIMessage, Tag(tag='ai')], typing.Annotated[langchain_core.messages.human.HumanMessage, Tag(tag='human')], typing.Annotated[langchain_core.messages.chat.ChatMessage, Tag(tag='chat')], typing.Annotated[langchain_core.messages.system.SystemMessage, Tag(tag='system')], typing.Annotated[langchain_core.messages.function.FunctionMessage, Tag(tag='function')], typing.Annotated[langchain_core.messages.tool.ToolMessage, Tag(tag='tool')], typing.Annotated[langchain_core.messages.ai.AIMessageChunk, Tag(tag='AIMessageChunk')], typing.Annotated[langchain_core.messages.human.HumanMessageChunk, Tag(tag='HumanMessageChunk')], typing.Annotated[langchain_core.messages.chat.ChatMessageChunk, Tag(tag='ChatMessageChunk')], typing.Annotated[langchain_core.messages.system.SystemMessageChunk, Tag(tag='SystemMessageChunk')], typing.Annota

In [89]:
formatted_chat_prompt = chat_prompt.format(
    word_count=5,
    conversation=[
        ("human", "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다."),
        ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
    ],
)

print(formatted_chat_prompt)

System: 당신은 요약 전문 AI 어시스턴트입니다. 당신의 임무는 주요 키워드로 대화를 요약하는 것입니다.
Human: 안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.
AI: 반가워요! 앞으로 잘 부탁 드립니다.
Human: 지금까지의 대화를 5 단어로 요약합니다.


In [90]:
chain = chat_prompt | llm | StrOutputParser()

In [94]:
# chain 실행 및 결과확인
chain.invoke(
    {
        "word_count": 5,
        "conversation": [
            (
                "human",
                "안녕하세요! 저는 오늘 새로 입사한 테디 입니다. 만나서 반갑습니다.",
            ),
            ("ai", "반가워요! 앞으로 잘 부탁 드립니다."),
        ],
    }
)

'새로 입사한 테디, 만남 반가워요.'